In [1]:
import cv2 as cv
import numpy as np

In [2]:
def showimage(image,label = "image"):
    cv.imshow(label,image)
    cv.waitKey()
    cv.destroyAllWindows()

In [3]:
video_handler = cv.VideoCapture('./Data/whiteline.mp4')
if (video_handler.isOpened() == False):
    print("Error opening the video file")
else:
# Get frame rate information
    fps = int(video_handler.get(5))
    print("Frame Rate : ",fps,"frames per second")	
    # Get frame count
    frame_count = video_handler.get(7)
    print("Frame count : ", frame_count)

frames = []
previous_tag=None
while(video_handler.isOpened()):
    # nonoise_vid.read() methods returns a tuple, first element is a bool 
    # and the second is frame
    ret, frame = video_handler.read()
    if ret == True:
        frames.append(frame)
        
    break

Frame Rate :  25 frames per second
Frame count :  221.0


In [4]:
showimage(frames[0])

In [68]:
gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
gray_blurred = cv.GaussianBlur(gray,(5,5),0)
clahe = cv.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
adpt_hist = clahe.apply(gray_blurred)
ret, thresh = cv.threshold(adpt_hist,127,256,cv.THRESH_BINARY+cv.THRESH_OTSU)
edges_image = cv.Canny(thresh,50,150,apertureSize = 3)
# showimage(thresh)
ROI_points = np.array([[(150,frame.shape[0]),(880,frame.shape[0]),(480,300)]])
masked_image = np.zeros_like(edges_image)
mask = cv.fillPoly(masked_image,ROI_points,255)
masked_image = cv.bitwise_and(edges_image,mask)
showimage(masked_image)

In [81]:
lines = cv.HoughLinesP(masked_image,1,np.pi/180,120,np.array([]),minLineLength=70,maxLineGap=5)

In [82]:
lines.shape

(4, 1, 4)

In [83]:
frame_cpy = np.copy(frame)

In [84]:
for i in range(0, len(lines)):
        l = lines[i][0]
        cv.line(frame_cpy, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)
        showimage(frame_cpy)

In [16]:
## Average out lengths the lines of the similar slope and use that to distinguish the lines 